# Creation and Application of Knowledge Graphs
# Introduction to Lab


This notebook is used for a small prologue to the lab exercises for the course "Creation and Applications of Knowledge Graphs"
For this course exercises, there are some prerequisites :
- Prior knowledge on Subject (Basics)
- Python 3
- Jupyter Notebook (Recap)

### Contents

- Jupyter (Quick Recap)

- Introduction of RDFLib & SPARQL

## Jupyter Notebook : Installation

Miniconda - https://docs.conda.io/projects/miniconda/en/latest/

### Jupyter Notebook

Just a quick look back on how it works. The markdown cell is used to type in documentation and uses 
[Markdown](https://www.markdownguide.org/cheat-sheet/) as markup. As obviously, code cell is for code execution and scripting

### Let's install 
To begin with let's install the RDFLib

In [1]:
from IPython.display import clear_output

In [2]:
!pip install rdflib SPARQLWrapper
# !conda install -y -c conda-forge rdflib SPARQLWrapper
clear_output()

### Let's start

First, let's get all the modules from the library

In [3]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib.namespace import RDF, RDFS
from SPARQLWrapper import SPARQLWrapper, JSON

### Creation of Instances

Now, since everything is set, let's create Namespace and Graph instance

In [4]:
#Creating a new Grpah
G = Graph()

# Required Namespaces
foaf = Namespace("http://xmlns.com/foaf/0.1/")
DBR = Namespace("http://dbpedia.org/resource/")
DBO = Namespace("http://dbpedia.org/ontology/")

### Adding Triples to Graph

Let's consider two persons, Anna and Micheal. Both of them belongs to the class 'Person'. First we create an URI Reference

In [5]:
person1 = URIRef("http://example.org/anna")
person2 = URIRef("http://example.org/Micheal")

# Adding nodes to graph
G.add((person1, RDF.type, DBO.Person))
G.add((person1, foaf.name, Literal("Anna")))

G.add((person2, RDF.type, DBO.Person))
G.add((person2, foaf.name, Literal("Micheal")))

<Graph identifier=N609787f3112241a69959c866d284bc50 (<class 'rdflib.graph.Graph'>)>

Let's say relation between two individuals are friendship. Let's consider that to graph and add the relation

In [6]:
G.add((person1, foaf.friendship, person2))

<Graph identifier=N609787f3112241a69959c866d284bc50 (<class 'rdflib.graph.Graph'>)>

### Saving the Graph

The Graph is serialized and save to storage for reuses. The graphs can be formatted in JSON, RDF/XML or Turtle syntax

In [7]:
G.serialize(destination="output.ttl", format="turtle")

<Graph identifier=N609787f3112241a69959c866d284bc50 (<class 'rdflib.graph.Graph'>)>

Now let's load the saved graph from the file

In [8]:
G2 = Graph().parse("output.ttl", format="ttl")

for s, p ,o in G2:
    print(f'{s} -> {p} -> {o}')

http://example.org/anna -> http://xmlns.com/foaf/0.1/friendship -> http://example.org/Micheal
http://example.org/anna -> http://xmlns.com/foaf/0.1/name -> Anna
http://example.org/anna -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://dbpedia.org/ontology/Person
http://example.org/Micheal -> http://xmlns.com/foaf/0.1/name -> Micheal
http://example.org/Micheal -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://dbpedia.org/ontology/Person


### The Query

Let's create a query based on previous example

In [9]:
# Create a Graph, parse from Internet data
g = Graph().parse("http://www.w3.org/People/Berners-Lee/card")

# Query the data in g using SPARQL
# This query returns the 'name' of all ``foaf:Person`` instances
q = """
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT ?name
    WHERE {
        ?p rdf:type foaf:Person .

        ?p foaf:name ?name .
    }
"""

# Apply the query to the graph and iterate through results
for r in g.query(q):
    print(r)
    print(r["name"])


(rdflib.term.Literal('Timothy Berners-Lee'),)
Timothy Berners-Lee


### SPARQL Wrapper

Let's say we need to search something. This is where SPARQL comes in hand. SPARQL is a query language for RDF graphs. RDF graphs are a way of representing data in a linked and interconnected way. SPARQL queries can be used to find specific information in RDF graphs, or to perform more complex tasks such as reasoning and inference.

Here's a example in general :
```
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?name
WHERE {
  ?person foaf:givenName ?name .
}
```

In [10]:
# Creating Instance
sparql = SPARQLWrapper(
    "https://dbpedia.org/sparql"
)
sparql.setReturnFormat(JSON)

# gets the spouse and birthdate of Joe Biden
# from the DBpedia database via a SPARQL endpoint

sparql.setQuery("""
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX xsd: <http://www.w3.org/2001/>

    SELECT ?spouse ?birthDate WHERE {
        dbr:Joe_Biden dbo:spouse ?spouse .
        ?spouse dbo:birthDate ?birthDate .
    }
    """
)

try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print(r)
except Exception as e:
    print(e)

{'spouse': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Jill_Biden'}, 'birthDate': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#date', 'value': '1951-06-03'}}
{'spouse': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Neilia_Hunter_Biden'}, 'birthDate': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#date', 'value': '1942-07-28'}}


## References

- Getting Started with RDFLib : https://rdflib.readthedocs.io/en/stable/gettingstarted.html
- SPARQL Wrapper : https://sparqlwrapper.readthedocs.io/en/latest/main.html#command-line-script
- Markdown Cheat Sheet : https://www.markdownguide.org/cheat-sheet/